In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=10,
    horizontal_flip=True,
    zoom_range=0.1,
    shear_range=0.1
)


#Create Traning Data
train_generator = train_datagen.flow_from_directory(
    'F:/Project/Final Dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

#Create Validation Data
validation_generator = train_datagen.flow_from_directory(
    'F:/Project/Final Dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 18677 images belonging to 2 classes.
Found 4668 images belonging to 2 classes.


In [8]:
base_model = tf.keras.applications.DenseNet121(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')


29084464/29084464 [==============================] - 9s 0us/step


In [9]:
# Freeze the base model layers
base_model.trainable = False

In [10]:
# Add custom layers on top of the base model for classification
model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [11]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [12]:
# Set up early stopping
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [13]:
history = model.fit(train_generator,
                    epochs=10,
                    validation_data=validation_generator,
                    callbacks=[early_stop])


Epoch 1/10
584/584 [==============================] - 863s 1s/step - loss: 0.2125 - accuracy: 0.9654 - val_loss: 0.0269 - val_accuracy: 0.9916
Epoch 2/10
584/584 [==============================] - 870s 1s/step - loss: 0.0597 - accuracy: 0.9788 - val_loss: 0.0152 - val_accuracy: 0.9942
Epoch 3/10
584/584 [==============================] - 867s 1s/step - loss: 0.0432 - accuracy: 0.9852 - val_loss: 0.0353 - val_accuracy: 0.9859
Epoch 4/10
584/584 [==============================] - 865s 1s/step - loss: 0.0407 - accuracy: 0.9848 - val_loss: 0.0133 - val_accuracy: 0.9964
Epoch 5/10
584/584 [==============================] - 896s 2s/step - loss: 0.0417 - accuracy: 0.9845 - val_loss: 0.0168 - val_accuracy: 0.9949
Epoch 6/10
584/584 [==============================] - 902s 2s/step - loss: 0.0404 - accuracy: 0.9860 - val_loss: 0.0176 - val_accuracy: 0.9957
Epoch 7/10
584/584 [==============================] - 879s 2s/step - loss: 0.0310 - accuracy: 0.9886 - val_loss: 0.0156 - val_accuracy: 0.9955

In [14]:
model.save('project17_DenseNetModel.h5')